In [1]:
import pandas as pd
import datautils
import modelutils as mu
import numpy as np
import itertools
import time
from tqdm import tqdm

## DATASET IMPORTING

In [2]:
DATASET = "Dataset/ML-CUP23-TR.csv"
RESULTS = "Results-chol/"
PLOT = "Plots/"
RUNS = "FullRuns/"

In [3]:
df_cup = pd.read_csv(DATASET, skiprows=6)
df_cup.rename(columns={"# Training set: ID": "ID"}, inplace=True)

In [4]:
df_cup.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      1000 non-null   int64  
 1   x1      1000 non-null   float64
 2   x2      1000 non-null   float64
 3   x3      1000 non-null   float64
 4   x4      1000 non-null   float64
 5   x5      1000 non-null   float64
 6   x6      1000 non-null   float64
 7   x7      1000 non-null   float64
 8   x8      1000 non-null   float64
 9   x9      1000 non-null   float64
 10  x10     1000 non-null   float64
 11  y1      1000 non-null   float64
 12  y2      1000 non-null   float64
 13  y3      1000 non-null   float64
dtypes: float64(13), int64(1)
memory usage: 109.5 KB


In [5]:
X_train, y_train = datautils.obtain_features_targets(df_cup)
print(X_train.shape, y_train.shape)

input_size = X_train.shape[1]
output_size = y_train.shape[1]
input_size, output_size

(1000, 10) (1000, 3)


(10, 3)

In [6]:
hidden_sizes = [*range(50, 1001, 50)]
alphas = [-1, -2, -3, -4, -5, -6]
seeds = range(4)

# Create parameter grid using itertools.product
param_grid = [
    {"Hidden size": hs, "Alpha": a} for hs, a in itertools.product(hidden_sizes, alphas)
]

In [7]:
results_dict = {
    "Hidden size": [],
    "Alpha": [],
    "Seed": [],
    "Soldist": [],
    "LSE": [],
    "LSE_Diff": [],
    "LSE_Diff_noalpha": [],
    "Total time": [],
    "Chol time": [],
    "Chol Residual": [],
    "QR Residual": [],
    "Rel_Soldist": [],
    "Rel_LSE_Diff": [],
    "Rel_LSE_Diff_noalpha": [],
}

In [8]:
# Wrap param_grid with tqdm for progress bar
for params in tqdm(param_grid, desc="Grid Search"):
    for seed in seeds:

        # Value of Alpha to use for experiment
        value = 10 ** params["Alpha"]

        # Initialize models
        model_qr = mu.ELM(input_size, params["Hidden size"], output_size, seed=seed)
        model_chol = mu.ELM(input_size, params["Hidden size"], output_size, seed=seed)

        # Compute QR solution
        model_qr.compute_wout_system_qr(X_train, y_train, alpha=value)

        # Use compute_wout_system, which internally uses cholesky decomposition
        start_total = time.process_time()
        chol_time = model_chol.compute_wout_system(X_train, y_train, alpha=value)
        total_time = time.process_time() - start_total

        # Compute residual for Cholesky system
        A = model_chol.hidden_activations(X_train)
        BtB = A.T @ A + value * np.eye(params["Hidden size"])
        Aty = A.T @ y_train
        chol_residual = np.linalg.norm(
            BtB @ model_chol.output_weights - Aty, ord="fro"
        ) / np.linalg.norm(Aty, ord="fro")

        # Compute residual for QR system
        A_qr = model_qr.hidden_activations(X_train)
        sqrt_alpha = np.sqrt(value)
        I = np.eye(model_qr.hidden_size)
        A_aug = np.vstack([A_qr, sqrt_alpha * I])
        Y_aug = np.vstack(
            [y_train, np.zeros((model_qr.hidden_size, model_qr.output_size))]
        )
        Q, R = np.linalg.qr(A_aug)
        qr_residual = np.linalg.norm(
            R @ model_qr.output_weights - Q.T @ Y_aug, ord="fro"
        ) / np.linalg.norm(Q.T @ Y_aug, ord="fro")

        # Compute metrics
        soldist = np.linalg.norm(
            model_chol.output_weights - model_qr.output_weights, "fro"
        )
        lse_chol = mu.compute_loss(
            y_train,
            model_chol.predict(X_train),
            model_chol.output_weights,
            alpha=value,
        )
        lse_qr = mu.compute_loss(
            y_train, model_qr.predict(X_train), model_qr.output_weights, alpha=value
        )
        lse_diff = abs(lse_chol - lse_qr)

        # Compute LSE difference without alpha
        lse_chol_noalpha = mu.compute_loss(
            y_train, model_chol.predict(X_train), model_chol.output_weights, alpha=0
        )
        lse_qr_noalpha = mu.compute_loss(
            y_train, model_qr.predict(X_train), model_qr.output_weights, alpha=0
        )
        lse_diff_noalpha = abs(lse_chol_noalpha - lse_qr_noalpha)

        # Compute relative gaps
        qr_wout_norm = np.linalg.norm(model_qr.output_weights, "fro")
        rel_soldist = soldist / qr_wout_norm if qr_wout_norm != 0 else np.nan
        rel_lse_diff = lse_diff / abs(lse_qr) if lse_qr != 0 else np.nan
        rel_lse_diff_noalpha = (
            lse_diff_noalpha / abs(lse_qr_noalpha) if lse_qr_noalpha != 0 else np.nan
        )

        # Store results
        results_dict["Hidden size"].append(params["Hidden size"])
        results_dict["Alpha"].append(value)
        results_dict["Seed"].append(seed)
        results_dict["Soldist"].append(soldist)
        results_dict["LSE"].append(lse_chol)
        results_dict["LSE_Diff"].append(lse_diff)
        results_dict["LSE_Diff_noalpha"].append(lse_diff_noalpha)
        results_dict["Total time"].append(total_time)
        results_dict["Chol time"].append(chol_time)
        results_dict["Chol Residual"].append(chol_residual)
        results_dict["QR Residual"].append(qr_residual)
        results_dict["Rel_Soldist"].append(rel_soldist)
        results_dict["Rel_LSE_Diff"].append(rel_lse_diff)
        results_dict["Rel_LSE_Diff_noalpha"].append(rel_lse_diff_noalpha)

results_df = pd.DataFrame(results_dict)
results_df.to_csv(RESULTS + "chol_experiment-diffalpha.csv", index=False)
results_df.head()

Grid Search: 100%|██████████| 120/120 [03:50<00:00,  1.92s/it]


,Hidden size,Alpha,Seed,Soldist,LSE,LSE_Diff,LSE_Diff_noalpha,Total time,Chol time,Chol Residual,QR Residual,Rel_Soldist,Rel_LSE_Diff,Rel_LSE_Diff_noalpha
0,50,0.10,0,7.556953e-10,38.514542,2.131628e-14,6.458833e-12,0.003243,0.001371,7.905152e-16,4.112499e-16,3.387098e-12,5.534606e-16,1.925898e-13
1,50,0.10,1,2.963871e-10,31.912069,7.105427e-15,1.264766e-12,0.002816,0.001154,5.125845e-16,4.746221e-16,1.482708e-12,2.226564e-16,4.530576e-14
2,50,0.10,2,4.551049e-10,38.279339,7.105427e-15,5.186962e-13,0.002912,0.001190,6.621535e-16,4.639201e-16,1.884090e-12,1.856204e-16,1.598712e-14
3,50,0.10,3,2.960072e-10,33.632197,7.105427e-15,1.740830e-13,0.002694,0.001151,5.875254e-16,4.296813e-16,1.381464e-12,2.112686e-16,5.994385e-15
4,50,0.01,0,1.544290e-09,32.457054,0.000000e+00,2.732037e-12,0.002864,0.001220,1.042224e-15,3.952573e-16,4.855026e-12,0.000000e+00,8.688220e-14


In [9]:
hidden_sizes = [3, 6, 10]
alphas = [-1, -2, -3, -4, -5, -6]
seeds = range(20)

# Create parameter grid using itertools.product
param_grid = [
    {"Hidden size": hs, "Alpha": a} for hs, a in itertools.product(hidden_sizes, alphas)
]

In [10]:
results_dict = {
    "Hidden size": [],
    "Alpha": [],
    "Seed": [],
    "Soldist": [],
    "LSE": [],
    "LSE_Diff": [],
    "LSE_Diff_noalpha": [],
    "Total time": [],
    "Chol time": [],
    "Chol Residual": [],
    "QR Residual": [],
    "Rel_Soldist": [],
    "Rel_LSE_Diff": [],
    "Rel_LSE_Diff_noalpha": [],
}

In [11]:
# Wrap param_grid with tqdm for progress bar
for params in tqdm(param_grid, desc="Grid Search"):
    for seed in seeds:
        value = 10 ** params["Alpha"]
        # Initialize models
        model_qr = mu.ELM(input_size, params["Hidden size"], output_size, seed=seed)
        model_chol = mu.ELM(input_size, params["Hidden size"], output_size, seed=seed)

        # Compute QR solution
        model_qr.compute_wout_system_qr(X_train, y_train, alpha=value)

        # Use compute_wout_system, which internally uses cholesky decomposition
        start_total = time.process_time()
        chol_time = model_chol.compute_wout_system(X_train, y_train, alpha=value)
        total_time = time.process_time() - start_total

        # Compute residual for Cholesky system
        A = model_chol.hidden_activations(X_train)
        BtB = A.T @ A + value * np.eye(params["Hidden size"])
        Aty = A.T @ y_train
        chol_residual = np.linalg.norm(BtB @ model_chol.output_weights - Aty, ord="fro")

        # Compute residual for QR system
        A_qr = model_qr.hidden_activations(X_train)
        sqrt_alpha = np.sqrt(value)
        I = np.eye(model_qr.hidden_size)
        A_aug = np.vstack([A_qr, sqrt_alpha * I])
        Y_aug = np.vstack(
            [y_train, np.zeros((model_qr.hidden_size, model_qr.output_size))]
        )
        Q, R = np.linalg.qr(A_aug)
        qr_residual = np.linalg.norm(
            R @ model_qr.output_weights - Q.T @ Y_aug, ord="fro"
        ) / np.linalg.norm(Q.T @ Y_aug, ord="fro")

        # Compute metrics

        # distance from solution
        soldist = np.linalg.norm(
            model_chol.output_weights - model_qr.output_weights, "fro"
        )

        # difference from qr LSE
        lse_chol = mu.compute_loss(
            y_train,
            model_chol.predict(X_train),
            model_chol.output_weights,
            alpha=value,
        )
        lse_qr = mu.compute_loss(
            y_train, model_qr.predict(X_train), model_qr.output_weights, alpha=value
        )
        lse_diff = abs(lse_chol - lse_qr)

        # LSE difference without alpha
        lse_chol_noalpha = mu.compute_loss(
            y_train, model_chol.predict(X_train), model_chol.output_weights, alpha=0
        )
        lse_qr_noalpha = mu.compute_loss(
            y_train, model_qr.predict(X_train), model_qr.output_weights, alpha=0
        )
        lse_diff_noalpha = abs(lse_chol_noalpha - lse_qr_noalpha)

        # Compute relative gaps
        qr_wout_norm = np.linalg.norm(model_qr.output_weights, "fro")
        rel_soldist = soldist / qr_wout_norm if qr_wout_norm != 0 else np.nan
        rel_lse_diff = lse_diff / abs(lse_qr) if lse_qr != 0 else np.nan
        rel_lse_diff_noalpha = (
            lse_diff_noalpha / abs(lse_qr_noalpha) if lse_qr_noalpha != 0 else np.nan
        )

        # Store results
        results_dict["Hidden size"].append(params["Hidden size"])
        results_dict["Alpha"].append(value)
        results_dict["Seed"].append(seed)
        results_dict["Soldist"].append(soldist)
        results_dict["LSE"].append(lse_chol)
        results_dict["LSE_Diff"].append(lse_diff)
        results_dict["LSE_Diff_noalpha"].append(lse_diff_noalpha)
        results_dict["Total time"].append(total_time)
        results_dict["Chol time"].append(chol_time)
        results_dict["Chol Residual"].append(chol_residual)
        results_dict["QR Residual"].append(qr_residual)
        results_dict["Rel_Soldist"].append(rel_soldist)
        results_dict["Rel_LSE_Diff"].append(rel_lse_diff)
        results_dict["Rel_LSE_Diff_noalpha"].append(rel_lse_diff_noalpha)

results_df = pd.DataFrame(results_dict)
results_df.to_csv(RESULTS + "chol_experiment-balanced.csv", index=False)
results_df.head()

Grid Search: 100%|██████████| 18/18 [00:00<00:00, 54.03it/s]


,Hidden size,Alpha,Seed,Soldist,LSE,LSE_Diff,LSE_Diff_noalpha,Total time,Chol time,Chol Residual,QR Residual,Rel_Soldist,Rel_LSE_Diff,Rel_LSE_Diff_noalpha
0,3,0.1,0,7.630502e-14,1584.811902,0.000000e+00,0.000000e+00,0.000112,0.000028,5.332385e-13,6.741196e-17,1.620567e-15,0.000000e+00,0.000000e+00
1,3,0.1,1,2.597994e-13,1292.454983,0.000000e+00,0.000000e+00,0.000094,0.000019,6.212338e-12,1.995549e-16,2.702039e-15,0.000000e+00,0.000000e+00
2,3,0.1,2,9.610915e-14,1108.455156,0.000000e+00,0.000000e+00,0.000068,0.000014,2.369082e-12,1.214678e-16,1.217826e-15,0.000000e+00,0.000000e+00
3,3,0.1,3,2.112633e-13,945.311781,2.273737e-13,2.273737e-13,0.000092,0.000017,4.336896e-12,6.006929e-17,2.781767e-15,2.405277e-16,2.406746e-16
4,3,0.1,4,1.338883e-13,1511.901775,0.000000e+00,0.000000e+00,0.000064,0.000012,1.652375e-12,7.517763e-17,1.790295e-15,0.000000e+00,0.000000e+00
